In [1]:
from sparq.tools.python_repl.python_repl_tool import python_repl_tool
from sparq.utils.helpers import get_llm
from sparq.settings import Settings

from pathlib import Path
import os

from langgraph.prebuilt import create_react_agent
from langchain.tools import tool

In [3]:
@tool
def my_tool_function(input: str) -> str:
    """
    A simple tool function that processes the input string and returns a formatted string.
    """
    return f"Processed input: {input}"

In [2]:
settings = Settings()
settings._load_env_variables(Path("../.env"))


if os.getenv("GEMINI_API_KEY"):
    model = "gemini-2.0-flash"
    provider = "google_genai"
    llm = get_llm(model=model, provider=provider)
else:
    raise ValueError("GEMINI_API_KEY is not set in the environment variables.")

In [4]:
try:
    agent = create_react_agent(
        model=llm,
        tools=[my_tool_function, python_repl_tool]
    )
    print("Agent created successfully with python REPL tool")
except Exception as e:
    print(f"Error creating agent: {e}")

Agent created successfully with python REPL tool


In [5]:
try:
    prompt = "Use python to add two numbers: 3.9345 and 5.4345"
    input = {"messages": [{"role": "user", "content": prompt}]}
    output = agent.invoke(input)
    print(f"Agent output: {output}")

    print("Type(output):", type(output))
    print("repr(output):", repr(output))
except Exception as e:
    print(f"Error running agent: {e}")

9.369
Agent output: {'messages': [HumanMessage(content='Use python to add two numbers: 3.9345 and 5.4345', additional_kwargs={}, response_metadata={}, id='d9e13b79-e110-4353-8d4d-0cddeb5749e5'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'python_repl_tool', 'arguments': '{"code": "print(3.9345 + 5.4345)"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--59c53471-a0f2-4e4f-8004-687eee498149-0', tool_calls=[{'name': 'python_repl_tool', 'args': {'code': 'print(3.9345 + 5.4345)'}, 'id': '6476824a-5c8b-4212-b1c3-2db057962bcd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 198, 'output_tokens': 23, 'total_tokens': 221, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='✓ Code executed successfully.\nOutput:\nNone', name='python_repl_tool', id='f0936cbd-515f-4725-b17b-8e110ec76396', tool_call_id='6476824a-5c8b-4212-b1c3-2

In [20]:
3.9345 + 5.4345

9.369

In [46]:
print("information on the output from agent.invoke()")
print("Type(output):", type(output))
print("repr(output):", repr(output))
print("len(output):", len(output))
print("output.keys():", output.keys())
print()
print("type(output['messages']):", type(output["messages"]))
print("len(output['messages']):", len(output["messages"]))
print(output['messages'])
print()
print("type(output['messages'][0]):", type(output["messages"][0]))
print("output['messages'][0].pretty_print():", output['messages'][0].pretty_print())
print()
print(output['messages'][1].pretty_print())

information on the output from agent.invoke()
Type(output): <class 'langgraph.pregel.io.AddableValuesDict'>
repr(output): {'messages': [HumanMessage(content='Use python to add two numbers: 3.9345 and 5.4345', additional_kwargs={}, response_metadata={}, id='3ad45a1b-a3ea-4775-bce9-10377b4be4e3'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'python_repl_tool', 'arguments': '{"code": "print(3.9345 + 5.4345)"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--2c49a18f-4730-4a01-96c4-a62101ec7c0b-0', tool_calls=[{'name': 'python_repl_tool', 'args': {'code': 'print(3.9345 + 5.4345)'}, 'id': '6ed6ab1a-04d4-498c-a4a7-651c1cb0f29d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 198, 'output_tokens': 23, 'total_tokens': 221, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='✓ Code executed successfully.\nOutput:\nNone', name=

In [51]:
output['messages'][1].additional_kwargs

{'function_call': {'name': 'python_repl_tool',
  'arguments': '{"code": "print(3.9345 + 5.4345)"}'}}

In [56]:
output['messages'][1].additional_kwargs['function_call']['arguments']

'{"code": "print(3.9345 + 5.4345)"}'

In [60]:
import json
print(json.loads(output['messages'][1].additional_kwargs['function_call']['arguments']))

# get the code
code = json.loads(output['messages'][1].additional_kwargs['function_call']['arguments'])['code']
print(code)

{'code': 'print(3.9345 + 5.4345)'}
print(3.9345 + 5.4345)


In [ ]:
# ============== Debug version with streaming ====================
import asyncio
from threading import Timer

# Create agent
agent = create_react_agent(
    model=llm,
    tools=[python_repl_tool]
)

# Get output with streaming to see what's happening
prompt = "Calculate the fibonacci of 1."
input_msg = {"messages": [{"role": "user", "content": prompt}]}

print("Starting agent execution...")
try:
    step_count = 0
    
    # Set a timeout for safety
    timeout_seconds = 30
    
    for chunk in agent.stream(input_msg, config={"recursion_limit": 5}):
        step_count += 1
        print(f"\n=== Step {step_count} ===")
        print(f"Chunk keys: {chunk.keys()}")
        
        # Print each message in the chunk
        for node_name, node_output in chunk.items():
            print(f"Node: {node_name}")
            if "messages" in node_output:
                for msg in node_output["messages"]:
                    msg_type = type(msg).__name__
                    print(f"  -> {msg_type}")
                    if hasattr(msg, "content"):
                        content = str(msg.content)[:500]
                        print(f"     Content: {content}")
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        print(f"     Tool calls: {msg.tool_calls}")
                    if hasattr(msg, "artifact"):
                        print(f"     Artifact: {msg.artifact}")
        
        if step_count >= 15:
            print("\n!!! Stopping - too many steps")
            break
            
except KeyboardInterrupt:
    print(f"\n!!! Interrupted by user at step {step_count}")
except Exception as e:
    print(f"\nError: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

print(f"\n=== Completed after {step_count} steps ===")

Starting agent execution...

=== Step 1 ===
Chunk keys: dict_keys(['agent'])

agent -> AIMessage
  Content: 
  Tool calls: [{'name': 'python_repl_tool', 'args': {'code': 'def fibonacci(n):\n    if n <= 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fibonacci(n-1) + fibonacci(n-2)\n\nprint(fibonacci(1))'}, 'id': 'a972a407-26e3-445e-87a4-ae2ac6c8dede', 'type': 'tool_call'}]
1
